In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cholladai_tokenizer/transformers/default/1/tokenizer.json
/kaggle/input/cholladai_tokenizer/transformers/default/1/tokenizer_config.json
/kaggle/input/cholladai_tokenizer/transformers/default/1/special_tokens_map.json
/kaggle/input/cholladai_tokenizer/transformers/default/1/vocab.txt
/kaggle/input/abs-dataset/dataset_final.csv


In [11]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification
tokenizer=AutoTokenizer.from_pretrained('/kaggle/input/cholladai_tokenizer/transformers/default/1')

In [3]:
tokenizer2=AutoTokenizer.from_pretrained('ai4bharat/indic-bert')

config.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/5.65M [00:00<?, ?B/s]

In [4]:
encode_2=tokenizer2.encode('இந்த மாதிரி வீடியோ போட்டா நா அடிப்பேன் செருப்பால')
print(encode_2)
for i in range(len(encode_2)):
    print(tokenizer2.decode(encode_2[i]))

[2, 1833, 13143, 3587, 6500, 5798, 7219, 1299, 5010, 5448, 4919, 22933, 3664, 122252, 39555, 3664, 39853, 3]
[CLS]
இ
ந
த
மத
ர
வட
ய
பட
ட
ந
அட
ப
பன
சர
ப
பல
[SEP]


In [9]:
encode=tokenizer.encode('இந்த மாதிரி வீடியோ போட்டா நா அடிப்பேன் செருப்பால')
print(encode)
for i in range(len(encode)):
    print(tokenizer.decode(encode[i]))

[2, 239, 2459, 2262, 27861, 1010, 16519, 30660, 42459, 3]
[CLS]
இநத
மாதிரி
வடியோ
போடடா
நா
அடிப
##பேன
செருபபால
[SEP]


In [12]:

model = AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased")

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

2025-06-23 02:32:22.049223: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750645942.244460      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750645942.299860      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
num_added_tokens = tokenizer.add_special_tokens({"pad_token": "[PAD]"})

model.resize_token_embeddings(len(tokenizer))

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(52000, 768, padding_idx=0)

In [14]:
import torch 
import torch.nn as nn
import torch.optim as optim 
from torch.utils.data import DataLoader,Dataset
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [15]:
criterion=nn.CrossEntropyLoss()
model.to(device)
optimizer=optim.AdamW(model.parameters(),lr=5e-5)

In [16]:
import pandas as pd
df=pd.read_csv('/kaggle/input/abs-dataset/dataset_final.csv')

In [17]:
df.head()

,Text,Class
0,குஷ்பூ அக்கா வாயை உண்மையிலேயே எட்டி உதைக்கணும்...,1
1,குஷ்பூ அக்கா வாயிலே நேரிலேயே எட்டி உதைக்க வேண்...,1
2,குஷ்பூ அக்கா வாயா உண்மையா எட்டி உதைக்க சொல்லுங...,1
3,குஷ்பூ அக்கா வாயில் உண்மையில் எட்டி உதைக்கணும்...,1
4,"""இவளின் யோனியில் கூட ஒரு மெட்ரோ ரயிலை ஓட்டலாம்...",1


In [18]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(
    df,
    test_size=0.2,
    random_state=42
)
test, valid = train_test_split(
    test,
    test_size=0.5,
    random_state=42
)

In [19]:
def tokenizer_text(example):
    return tokenizer(example,padding='max_length',truncation=True,max_length=120,return_tensors='pt')

In [20]:
class TextDataset(Dataset):
    def __init__(self,texts,labels):
        self.encoding=tokenizer_text(texts)
        self.label=torch.tensor(labels)
    def __len__(self):
        return len(self.label)
    def __getitem__(self,idx):
        return {
            'input_ids':self.encoding['input_ids'][idx],
            'attention_mask':self.encoding['attention_mask'][idx],
            'labels':self.label[idx],
        }

In [21]:
train_dataset=TextDataset(train['Text'].tolist(),train['Class'].tolist())
test_dataset=TextDataset(test['Text'].tolist(),test['Class'].tolist())

In [22]:
train_loader=DataLoader(train_dataset,batch_size=30,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=30,shuffle=True)

In [23]:
from tqdm import tqdm
from transformers import get_scheduler
num_training_steps=len(train_loader)*3
lr_scheduler=get_scheduler("linear",optimizer=optimizer,num_warmup_steps=1000,num_training_steps=num_training_steps)
epoch=3
for i in range(epoch):
    model.train()
    total_loss=0
    for batch in tqdm(train_loader):
        batch={key:val.to(device) for key,val in batch.items()}
        optimizer.zero_grad()
        output=model(**batch)
        loss=output.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        total_loss+=loss.item()
    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}, Training Loss: {avg_train_loss:.4f}")
        

100%|██████████| 993/993 [03:03<00:00,  5.41it/s]


Epoch 4, Training Loss: 0.4894


100%|██████████| 993/993 [03:02<00:00,  5.43it/s]


Epoch 4, Training Loss: 0.2863


100%|██████████| 993/993 [03:02<00:00,  5.43it/s]

Epoch 4, Training Loss: 0.1588


In [24]:
from sklearn.metrics import classification_report

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        batch = {key: val.to(device) for key, val in batch.items()}
        outputs = model(**batch)
        predictions = torch.argmax(outputs.logits, dim=-1)
        
        all_preds.extend(predictions.cpu().numpy())
        all_labels.extend(batch["labels"].cpu().numpy())

accuracy = (np.array(all_preds) == np.array(all_labels)).sum() / len(all_labels)
print(f"Validation Accuracy: {accuracy:.4f}")
print(classification_report(all_labels, all_preds, digits=4))

Validation Accuracy: 0.8697
              precision    recall  f1-score   support

           0     0.8830    0.8644    0.8736      1939
           1     0.8559    0.8756    0.8656      1784

    accuracy                         0.8697      3723
   macro avg     0.8695    0.8700    0.8696      3723
weighted avg     0.8700    0.8697    0.8698      3723

